In [28]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [71]:
excel_file_path = 'all_data.xlsx'
csv_file_path = 'all_data.csv'
df = pd.read_excel(excel_file_path, sheet_name=0)
# 削除する列のリスト
columns_to_drop = ['Density [kg/m^3]', '緯度', '経度', 'DO eq', '備考', '水深', 'pH', 'Pressure', 'O2 weiss 1970','DIC[μmol/L@25℃]','Depth[m]', 'DO[ µmol/kg]', 'TA [μmol/kg]','Salinity']
df.drop(columns=columns_to_drop, inplace=True)
df = df[df['label'] != -1]
# CSVファイルとして保存
df.to_csv(csv_file_path, index=False)
print(f"CSVファイルが保存されました: {csv_file_path}")
df

CSVファイルが保存されました: all_data.csv


,No.,pCO2,AOU,excess DIC,T,nDIC+0.768DO,DIC-0.5TA+0.83DO,場所,日時,DO Saturation,label
18,19,609.054405,5.103117,65.754199,26.1010,2234.610383,1053.148478,橘湾温泉,2020-08-31,0.975200,1
135,136,564.930029,12.868835,49.888406,18.0400,2285.756471,1132.068050,橘湾温泉,2024-05-21,0.945459,1
136,137,566.704024,16.336327,50.400634,18.0700,2283.360541,1129.289180,橘湾温泉,2024-05-22,0.930744,1
137,138,569.721742,8.624603,51.355938,18.2300,2288.770308,1135.138190,橘湾温泉,2024-05-23,0.963304,1
138,139,568.501477,12.060292,50.985970,18.1700,2286.217390,1132.425550,橘湾温泉,2024-05-24,0.948760,1
...,...,...,...,...,...,...,...,...,...,...,...
2060,2061,542.771000,118.587515,49.811000,10.6016,2265.132400,1143.534000,北大西洋,NaT,0.521004,0
2061,2062,557.977000,130.831224,53.484000,10.1420,2265.232200,1141.832000,北大西洋,NaT,0.483192,0
2062,2063,519.493000,119.491733,43.128000,10.7688,2255.845200,1134.107000,北大西洋,NaT,0.516312,0
2063,2064,538.981000,128.104189,48.153000,10.0288,2263.360400,1140.654000,北大西洋,NaT,0.493668,0


## EC-AO, CAO-T, NCO-T

In [ ]:
def analyze_with_prediction_interval(df, param1, param2, indicator, unit, figsize=(16, 8)):
    # label = 0 (漏出）のデータのみを抽出して回帰分析を行う
    df_normal = df[df['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(x.flatten(), y)

    # 回帰直線の予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    y_pred = slope * x_line.flatten() + intercept

    # 予測区間の計算
    # 信頼度95%の予測区間を計算
    pi = 0.95

    # 予測区間の計算に必要な統計量
    n = len(x)  # label = 1 のデータ数
    x_mean = np.mean(x)

    # 標準誤差の計算
    sum_sq_err = np.sum((y - (slope * x.flatten() + intercept))**2)
    std_err = np.sqrt(sum_sq_err / (n-2))

    # 予測区間の計算
    x_new = x_line.flatten()

    # t統計量
    t = stats.t.ppf((1 + pi) / 2, n-2)

    # 予測区間の幅を計算
    pi_width = t * std_err * np.sqrt(1 + 1/n +
                                    (x_new - np.mean(x.flatten()))**2 /
                                    np.sum((x.flatten() - np.mean(x.flatten()))**2))

    # 上側予測区間
    upper_pi = y_pred + pi_width

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("white")

    # 実データのプロット
    actual_leak = df[df['label'] == 1]
    plt.scatter(actual_leak[param2],
               actual_leak[param1],
               color='red',
               marker='o',
               label='Actual Leak',
               alpha=1,
               s=50)

    actual_non_leak = df[(df['label'] == 0) & (df['場所'].isin(['苫小牧','オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋']))]
    plt.scatter(actual_non_leak[param2],
               actual_non_leak[param1],
               color='black',
               marker='x',
               label='Actual Non-Leak',
               alpha=0.3,
               s=50)

    # 回帰直線と予測区間のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Regression Line (y = {slope:.3f}x + {intercept:.3f})')
    plt.plot(x_line, upper_pi, 'r--', label='95% Upper Prediction Interval')

    # グラフの設定
    plt.xlabel(f'{param2}({unit})', fontsize=16)
    plt.ylabel(f'{param1}(μmol/kg)', fontsize=16)
    plt.title(f'Seepage judgement by {indicator}', fontsize=20)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=16)

    plt.tight_layout()
    plt.savefig(f'accuracy_rate_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 95%上側予測区間を超えるデータポイントを特定
    # 各データポイントの予測値を計算（全データに対して）
    x_all = df[param2].values.reshape(-1, 1)
    y_pred_data = slope * x_all.flatten() + intercept

    # 各データポイントの予測区間を計算
    pi_width_data = t * std_err * np.sqrt(1 + 1/n +
                                         (x_all.flatten() - np.mean(x.flatten()))**2 /
                                         np.sum((x.flatten() - np.mean(x.flatten()))**2))
    upper_pi_data = y_pred_data + pi_width_data

    # 予測区間を超えるデータポイントを特定
    outliers = df[df[param1] > upper_pi_data]

    # ラベルごとの内訳（No.のリスト）
    outliers_actual_leak = outliers[outliers['label'] == 1]['No.'].tolist()
    outliers_actual_non_leak = outliers[(outliers['label'] == 0) & (outliers['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋']))]['No.'].tolist()

    # label = -1のデータ数と割合の計算
    total_leak_data = len(df[df['label'] == 1])
    total_no_leak_data = len(df[(df['label'] == 0) & (df['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋']))])
    outliers_leak_count = len(outliers_actual_leak)
    outliers_no_leak_count = len(outliers_actual_non_leak)
    leak_outlier_percentage = (100 - (total_leak_data - outliers_leak_count + outliers_no_leak_count) / (total_leak_data + total_no_leak_data) * 100) if total_leak_data + total_no_leak_data > 0 else 0
    precision = outliers_leak_count / (outliers_leak_count + outliers_no_leak_count) * 100
    recall = outliers_leak_count/ 28 *100
    f1_score = (2 * precision/100 * recall/100)/ (precision/100 + recall/100)

    print(f"\n95%上側予測区間を超えるデータポイントの分析:")
    print(f"\n統計情報:")
    print(f"- 予測区間を超える漏出データ数: {outliers_leak_count}")
    print(f"- 正解率: {leak_outlier_percentage:.2f}%")
    print(f"- precision: {precision:.2f}%")
    print(f"- recall: {recall:.2f}%")
    print(f"- f1_score: {f1_score:.2f}%")
    print(f"- 内訳:")
    print(f"  - 漏出(label=1): No.{outliers_actual_leak}")
    print(f"  - 非漏出(label=0): No.{outliers_actual_non_leak}")
    print(f"決定係数(R²): {r_value**2:.3f}")

    return outliers

## PC-OS

In [73]:
def analyze_with_prediction_interval_pCO2_DOsaturation(df, param1, param2, indicatior, figsize=(16, 8)):
    df_normal = df[df['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋'])].copy()
    x = df_normal[param2].values.reshape(-1, 1)
    y = df_normal[param1].values

    # 対数変換
    log_x = np.log(x)
    log_y = np.log(y)

    # 対数空間で線形回帰を実行
    slope, intercept, r_value, p_value, std_err = stats.linregress(log_x.flatten(), log_y)

    # 累乗関数のパラメータに変換
    a = np.exp(intercept)  # 係数
    b = slope  # 指数

    # 予測値を計算（全データ範囲で）
    x_line = np.linspace(df[param2].min(), df[param2].max(), 100).reshape(-1, 1)
    # 累乗関数での予測値
    y_pred = a * (x_line ** b).flatten()

    # 予測区間の計算（対数空間で）
    pi = 0.95
    n = len(x)
    log_x_mean = np.mean(log_x)

    # 対数空間での標準誤差
    log_y_pred = slope * log_x.flatten() + intercept
    sum_sq_err = np.sum((log_y - log_y_pred)**2)
    std_err = np.sqrt(sum_sq_err / (n-2))

    # t統計量
    t = stats.t.ppf((1 + pi) / 2, n-2)

    # 対数空間での予測区間
    log_x_new = np.log(x_line.flatten())
    pi_width = t * std_err * np.sqrt(1 + 1/n +
                                    (log_x_new - log_x_mean)**2 /
                                    np.sum((log_x.flatten() - log_x_mean)**2))

    # 予測区間を元のスケールに戻す
    log_y_pred_line = slope * log_x_new + intercept
    upper_pi = np.exp(log_y_pred_line + pi_width)

    # プロットの作成
    plt.figure(figsize=figsize)
    sns.set_style("white")

    # 実データのプロット
    actual_leak = df[df['label'] == 1]
    plt.scatter(actual_leak[param2],
               actual_leak[param1],
               color='red',
               marker='o',
               label='Actual Leak',
               alpha=1,
               s=50)

    actual_non_leak = df[(df['label'] == 0) & (df['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋']))]
    plt.scatter(actual_non_leak[param2],
               actual_non_leak[param1],
               color='black',
               marker='x',
               label='Actual Non-Leak',
               alpha=0.3,
               s=50)

    # 累乗回帰曲線と予測区間のプロット
    plt.plot(x_line, y_pred, 'b-', label=f'Power Regression (y = {a:.2f}x^{b:.2f})')
    plt.plot(x_line, upper_pi, 'r--', label='95% Upper Prediction Interval')

    # グラフの設定
    plt.xlabel(f'{param2}', fontsize=16)
    plt.ylabel(f'{param1}(μatm)', fontsize=16)
    plt.title(f'Seepage judgement by {indicatior}', fontsize=20)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=16)

    plt.tight_layout()
    plt.savefig(f'accuracy_rate_{param1}_{param2}.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 95%上側予測区間を超えるデータポイントを特定
    x_all = df[param2].values.reshape(-1, 1)
    log_x_all = np.log(x_all)

    # 対数空間での予測値と予測区間
    log_y_pred_data = slope * log_x_all.flatten() + intercept
    pi_width_data = t * std_err * np.sqrt(1 + 1/n +
                                         (log_x_all.flatten() - log_x_mean)**2 /
                                         np.sum((log_x.flatten() - log_x_mean)**2))

    # 元のスケールに戻す
    upper_pi_data = np.exp(log_y_pred_data + pi_width_data)

    # 予測区間を超えるデータポイントを特定
    outliers = df[df[param1] > upper_pi_data]

    # ラベルごとの内訳（No.のリスト）
    outliers_actual_leak = outliers[outliers['label'] == 1]['No.'].tolist()
    outliers_actual_non_leak = outliers[(outliers['label'] == 0) & (outliers['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋']))]['No.'].tolist()
    total_leak_data = len(df[df['label'] == 1])
    total_no_leak_data = len(df[(df['label'] == 0) & (df['場所'].isin(['苫小牧', 'オーストラリア北東','インドネシア','オーストラリア南東','マダガスカル', 'ギアナ沖', '北大西洋']))])
    outliers_leak_count = len(outliers_actual_leak)
    outliers_no_leak_count = len(outliers_actual_non_leak)
    leak_outlier_percentage = (100 - (total_leak_data - outliers_leak_count + outliers_no_leak_count) / (total_leak_data + total_no_leak_data) * 100) if total_leak_data + total_no_leak_data > 0 else 0
    precision = outliers_leak_count / (outliers_leak_count + outliers_no_leak_count) * 100
    recall = outliers_leak_count/ 28 *100
    f1_score = (2 * precision/100 * recall/100)/ (precision/100 + recall/100)


    print(f"\n95%上側予測区間を超えるデータポイントの分析:")
    print(f"\n統計情報:")
    print(f"回帰式: y = {a:.2f}x^{b:.2f}")
    print(f"- 正解率: {leak_outlier_percentage:.2f}%")
    print(f"- precision: {precision:.2f}%")
    print(f"- recall: {recall:.2f}%")
    print(f"- f1_score: {f1_score:.2f}%")
    print(f"- 予測区間を超える漏出データ数: {outliers_leak_count}")
    print(f"- 内訳:")
    print(f"  - 漏出(label=1): No.{outliers_actual_leak}")
    print(f"  - 非漏出(label=0): No.{outliers_actual_non_leak}")
    print(f"決定係数(R²): {r_value**2:.3f}")
    return outliers

## Create Graph

In [74]:
outliers = analyze_with_prediction_interval_pCO2_DOsaturation(df, 'pCO2', 'DO Saturation', 'PC-OS')
outliers = analyze_with_prediction_interval(df, 'excess DIC', 'AOU', 'EC-AO', 'μmol/kg')
outliers = analyze_with_prediction_interval(df, 'nDIC+0.768DO', 'T', 'NCO-T', '°C')
outliers = analyze_with_prediction_interval(df, 'DIC-0.5TA+0.83DO', 'T', 'CAO-T', '°C')


95%上側予測区間を超えるデータポイントの分析:

統計情報:
回帰式: y = 354.67x^-0.59
- 正解率: 97.67%
- precision: 38.81%
- recall: 92.86%
- f1_score: 0.55%
- 予測区間を超える漏出データ数: 26
- 内訳:
  - 漏出(label=1): No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 159, 160, 161, 162, 213, 214]
  - 非漏出(label=0): No.[299, 300, 302, 303, 343, 347, 915, 941, 1183, 1307, 1332, 1333, 1351, 1442, 1473, 1483, 1491, 1492, 1542, 1544, 1546, 1549, 1552, 1571, 1579, 1583, 1588, 1591, 1593, 1596, 1597, 1598, 1599, 1601, 1602, 1603, 1607, 1609, 1615, 1616, 1617]
決定係数(R²): 0.934

95%上側予測区間を超えるデータポイントの分析:

統計情報:
- 予測区間を超える漏出データ数: 24
- 正解率: 96.70%
- precision: 29.63%
- recall: 85.71%
- f1_score: 0.44%
- 内訳:
  - 漏出(label=1): No.[19, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 159, 160, 161, 162, 214]
  - 非漏出(label=0): No.[253, 303, 687, 703, 748, 915, 1332, 1333, 1341, 1343, 1351, 1352, 1356, 1357, 1374, 1379, 1380, 1387, 1393, 1401, 1410, 1411, 1415,

## define_accuracy.xlsx -> combining 4 indicators (if a datapoint exceeded 95% upper prediction interval in all 4 indicator, it will be classified as seepage)